In [141]:
import json
import nltk # stemmer
import numpy as np
import pandas as pd
import patsy
import re
import sklearn as skl
import string

# Preprocess data

In [5]:
train = pd.read_json("data/music_200.json", lines=True)
test = pd.read_json("data/music_test_200.json", lines=True)
train.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [6]:
train = train.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
test = test.drop(['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime'], axis=1)
train.head()

,overall,reviewText
0,5,"Not much to write about here, but it does exac..."
1,5,The product does exactly as it should and is q...
2,5,The primary job of this device is to block the...
3,5,Nice windscreen protects my MXL mic and preven...
4,5,This pop filter is great. It looks and perform...


In [196]:
train2 = train.assign(good = lambda g: g.overall >= 4)
trainFinal = train.assign(good = train2['good'].apply(lambda g: 1 if g else 0))

In [184]:
def rm_stopwords_punctuation(text):
    text = text.lower()
    with open("stopwords.json") as stopword_file:
        stopwords = json.load(stopword_file)
        for word in stopwords:
            if word in text:
                # replace only complete words ('\b' is a word boundary)
                text = re.sub(r"\b{}\b".format(word), "", text)
    # remove punctuation
    for char in string.punctuation:
        text = text.replace(char, "")
    text = re.sub(r"\b[a-z]\b", "", text)
    # remove whitespace
    for char in string.punctuation:
        text = text.replace(char, "")
    text = ' '.join(text.split(None))
    return text

In [185]:
def stem(text):
    stemmer = nltk.stem.porter.PorterStemmer()
    # stem each word individually, and concatenate
    text = ' '.join([stemmer.stem(word) for word in text.split(None)])
    return text

In [186]:
def process_text(text):
    text = rm_stopwords_punctuation(text)
    text = stem(text)
    return text

In [187]:
trainFinal['reviewText'] = trainFinal['reviewText'].apply(lambda t: process_text(t))

In [200]:
test2 = test.assign(good = lambda g: g.overall >= 4)
testFinal = test.assign(good = test2['good'].apply(lambda g: 1 if g else 0))
testFinal['reviewText'] = testFinal['reviewText'].apply(lambda t: process_text(t))

In [201]:
y, X = patsy.dmatrices("good ~ reviewText", trainFinal, return_type="dataframe")
y_test, X_test = patsy.dmatrices("good ~ reviewText", testFinal, return_type="dataframe")

In [202]:
print(y)
print(len(y), len(X))

     good
0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     0.0
8     1.0
9     1.0
10    1.0
11    1.0
12    0.0
13    1.0
14    1.0
15    0.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
..    ...
170   1.0
171   1.0
172   1.0
173   1.0
174   1.0
175   1.0
176   1.0
177   1.0
178   1.0
179   0.0
180   1.0
181   1.0
182   1.0
183   1.0
184   1.0
185   1.0
186   1.0
187   1.0
188   1.0
189   1.0
190   1.0
191   1.0
192   1.0
193   1.0
194   1.0
195   1.0
196   1.0
197   1.0
198   1.0
199   1.0

[200 rows x 1 columns]
200 200


# Logistic Regression

In [203]:
logRegrModel = skl.linear_model.LogisticRegression()
logRegrModel = logRegrModel.fit(X, y['good'])

In [204]:
logRegrModel.score(X, y['good'])

0.92000000000000004

In [205]:
logRegrModel.score(X_test, y_test['good'])

0.89000000000000001